In [1]:
import json

In [2]:
train_ocr_path = "/root/QianZe/Transformers-VQA/data/vizwiz/ocr_result/train_ocr_result_en_PP-ocrv3.json"
val_ocr_path = "/root/QianZe/Transformers-VQA/data/vizwiz/ocr_result/val_ocr_result_en_PP-ocrv3.json"

In [3]:
def read_ocr(path):
    with open(path, "r") as f:
        data = json.load(f)
    
    return data

In [4]:
def process_single_image(boxes):
    if len(boxes) == 0:
        return -1
    
    ratio = 0
    for box in boxes:
        pos, text = box
        x1, x2, x3, x4 = pos
        wide = x2[0] - x1[0]
        height = x3[1] - x2[1]
        if wide == 0.0:
            wide = 1
        if height == 0.0:
            height = 1
        tmp_ratio = wide / height
#         print(tmp_ratio)
        ratio += tmp_ratio
    
    average_ratio = ratio / len(boxes)
    
    return average_ratio

In [5]:
def process_images(ocr_data):
    ratio_dict = {}
    for image_id, boxes in ocr_data.items():
        average_ratio = process_single_image(boxes)
        ratio_dict[image_id] = average_ratio
    
    return ratio_dict

In [6]:
def count(ratio_dict, h_thres, v_thres):
    h_cnt, v_cnt, no_ocr = 0, 0, 0
    for ratio in list(ratio_dict.values()):
        if ratio == -1:
            no_ocr += 1
        elif ratio >= h_thres:
            h_cnt += 1
        elif ratio < v_thres:
            v_cnt += 1
    
    return h_cnt, v_cnt, no_ocr

In [7]:
def extract_rects(ocr_info):
    rects = []
    for boxes in ocr_info:
        rect, text = boxes
        rects.append(np.array(rect))
    
    return rects

In [8]:
train_ocr = read_ocr(train_ocr_path)
train_ocr_ratio = process_images(train_ocr)

In [9]:
val_ocr = read_ocr(val_ocr_path)
val_ocr_ratio = process_images(val_ocr)

In [10]:
h_threshold = 2
v_threshold = 0.5

In [11]:
count(val_ocr_ratio, h_threshold, v_threshold)
# count(train_ocr_ratio, h_threshold, v_threshold)

(3033, 551, 3613)

In [12]:
test_image = "/root/QianZe/VizWiz-VQA/val/VizWiz_val_00000031.jpg"
test_rects = extract_rects(val_ocr["VizWiz_val_00000031"])

In [17]:
test_rects = np.array(test_rects)

In [19]:
test_rects

array([[[209., 815.],
        [339., 846.],
        [332., 874.],
        [202., 843.]],

       [[345., 865.],
        [486., 895.],
        [481., 918.],
        [340., 888.]],

       [[194., 879.],
        [598., 960.],
        [594., 979.],
        [190., 898.]]])

In [14]:
def get_rect_points(text_boxes):
    x1 = np.min(text_boxes[:, :, 0])
    y1 = np.min(text_boxes[:, :, 1])
    x2 = np.max(text_boxes[:, :, 0])
    y2 = np.max(text_boxes[:, :, 1])

    return [x1, y1, x2, y2]

In [18]:
get_rect_points(test_rects)

[190.0, 815.0, 598.0, 979.0]